## RAG Langchain Q&A Pipeline

### Import packages

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter  
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Weaviate
from langchain.agents import AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.embeddings import OpenAIEmbeddings
from typing import List,Union
from weaviate.embedded import EmbeddedOptions
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv

In [ ]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from rag_utils import data_loader, create_retriever
from langchain_utils import get_agent_executor

### Load the data and create chunks

In [ ]:
chuncks = data_loader("../prompts/context.txt")
len(chuncks)

### Create the retriver

In [ ]:
retriever = create_retriever(chuncks)

In [ ]:
type(retriever)

In [ ]:
retriever.get_relevant_documents("Who are the parties to the Agreement and what are their defined names?")

### Create langchain executor agent

In [ ]:
llm_agent = get_agent_executor(retriever)

In [ ]:
llm_agent.run("Who are the parties to the Agreement and what are their defined names?")

In [ ]:
llm_agent.run("What is the termination notice?")

In [ ]:
llm_agent.run("What are the payments to the Advisor under the Agreement?")

In [ ]:
llm_agent.run("Is the Advisor entitled to social benefits?")